# --- Day 3: Crossed Wires ---

## ---Part One---

The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the <u src=https://en.wikipedia.org/wiki/Taxicab_geometry> Manhattan distance </u> for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is ```R8,U5,L5,D3```, then starting from the central port ```(o)```, it goes right 8, up 5, left 5, and finally down 3:

```...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........```

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

```...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........```

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

**Here are a few more examples:**
```
R8,U5,L5,D3
U7,R6,D4,L4
distance 6

R75,D30,R83,U83,L12,D49,R71,U7,L72<br>
U62,R66,U55,R34,D71,R55,D58,R83<br>
distance 159

R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51<br>
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7<br>
distance 135
```
**What is the Manhattan distance from the central port to the closest intersection?**

In [1]:
def get_points(path):
    x,y = [0,0]
    wire = []
    for move in path:
        if move[0] == 'L':
            wire += [(x-i,y) for i in range(1,int(move[1:])+1)]
            x -= int(move[1:])
        if move[0] == 'R':
            wire += [(x+i,y) for i in range(1,int(move[1:])+1)]
            x += int(move[1:])
        if move[0] == 'U':
            wire += [(x,y+i) for i in range(1,int(move[1:])+1)]
            y += int(move[1:])
        if move[0] == 'D':
            wire += [(x,y-i) for i in range(1,int(move[1:])+1)]
            y -= int(move[1:])
    return wire

In [2]:
def get_closest_intersect(wire1, wire2):
    intersects = set(wire1).intersection(wire2)
    return min([(abs(point[0]) + abs(point[1])) for point in intersects])

In [3]:
test1 = [['R8','U5','L5','D3'], ['U7','R6','D4','L4']]
wires1 = [get_points(i) for i in test1]
get_closest_intersect(wires1[0], wires1[1])

6

In [4]:
test2 = [['R75','D30','R83','U83','L12','D49','R71','U7','L72'], ['U62','R66','U55','R34','D71','R55','D58','R83']]
wires2 = [get_points(i) for i in test2]
get_closest_intersect(wires2[0], wires2[1])

159

In [5]:
test3 = [['R98','U47','R26','D63','R33','U87','L62','D20','R33','U53','R51'], ['U98','R91','D20','R16','D67','R40','U7','R15','U6','R7']]
wires3 = [get_points(i) for i in test3]
get_closest_intersect(wires3[0], wires3[1])

135

In [6]:
with open('day3.txt') as f:
    paths = [ line.strip().split(',') for line in f ]
    route = [ get_points(wire) for wire in paths ]
    print(get_closest_intersect(route[0], route[1]))

227


## --- Part Two ---

It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

`...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........`

***In the above example the intersection closest to the central port is reached after:***

* 8+5+5+2 = 20 steps by the first wire
* 7+6+4+3 = 20 steps by the second wire 
* for a total of 20+20 = 40 steps.

***However, the top-right intersection is better:***
* the first wire takes only 8+5+2 = 15 
* the second wire takes only 7+6+2 = 15
* for a total of 15+15 = 30 steps.

***Here are the best steps for the extra examples from above:***

R75,D30,R83,U83,L12,D49,R71,U7,L72,<br>
U62,R66,U55,R34,D71,R55,D58,R83 = 610 steps<br>

R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51<br>
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = 410 steps<br>

**What is the fewest combined steps the wires must take to reach an intersection?**

In [7]:
def get_shortest_intersect(wire1, wire2):
    intersects = set(wire1).intersection(wire2)
    steps = [ (wire1.index(loc), wire2.index(loc)) for loc in intersects ]
    return min([ step[0] + step[1] for step in steps ]) + 2

In [8]:
get_shortest_intersect(wires1[0], wires1[1])

30

In [9]:
get_shortest_intersect(wires2[0], wires2[1])

610

In [10]:
get_shortest_intersect(wires3[0], wires3[1])

410

In [11]:
with open('day3.txt') as f:
    paths = [ line.strip().split(',') for line in f ]
    route = [ get_points(wire) for wire in paths ]
    print(get_shortest_intersect(route[0], route[1]))

20286
